In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [53]:
# Get the number of last page avalaible
def get_last_news_page() -> int:
    url = "https://www.gamespot.com/news/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    btn = soup.find_all('a', class_='btn', recursive=True)
    pages = []
    for b in btn:
        try:
            p = int(b.text)
            pages.append(p)
        except:
            continue
    last_page = max(pages)
    return last_page

In [46]:
# Get all news from first_page to the last_page. Returns array of dictionaries where each of them has info about 100 pages of news
def get_all_gamespot_news(last_page: int, first_page: int = 1) -> list[dict]:    
    dict_array = []
    for i in range(last_page):
        # if i == 202:
            # break
        if i%100 == 0:
            titles, links, dates, pages = [],[],[],[]
            if i != 0:
                final_dict = {}
                final_dict['titles'] = titles
                final_dict['links'] = links
                final_dict['dates'] = dates
                final_dict['pages'] = pages
                dict_array.append(final_dict)
        try:
            response = requests.get(url + "?page=" + str(last_page - i))
            soup = BeautifulSoup(response.text, 'html.parser')
            news = soup.find_all('div', class_="card-item__content")
            for i in range(len(news)):
                a = news[i].find('a', class_='card-item__link')
                title = a.text
                link = "https://www.gamespot.com" + str(a.get('href'))
                date = news[i].find('time').get('datetime')
                titles.append(title)
                links.append(link)
                dates.append(date)
                pages.append(last_page - i)
        except ConnectionError as e:
            print(e)
        if last_page - i == first_page:
            break
    return dict_array


In [47]:
dict_array = get_all_gamespot_news(get_last_news_page())
for i, d in enumerate(dict_array):
    pages = dict_array[i]['pages']
    del dict_array[i]['pages']
    df = pd.DataFrame(d)
    df.to_csv("Data/Result/pages" + str(max(pages)) + "-" + str(min(pages)) + ".csv")